# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Store Part I results into DataFrame

weather_data = pd.read_csv("output_data/cities.csv")
weather_data.head()


,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Tuktoyaktuk,69.4541,-133.0374,53.15,56,57,12.46,CA,1627077620
1,Lyman,48.9884,37.8023,61.88,59,0,3.94,UA,1627077621
2,Stranda,62.3096,6.9358,60.35,91,97,2.01,NO,1627077622
3,Punta Arenas,-53.1500,-70.9167,37.11,86,57,18.03,CL,1627077622
4,Aklavik,68.2191,-135.0107,63.43,72,75,4.61,CA,1627077544


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=100, 
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 70 and 80
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.


narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Clouds"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
48,Jadu,31.9530,12.0261,74.59,41,0,7.96,LY,1627077642
171,Zhezkazgan,47.8043,67.7144,79.02,30,0,6.38,KZ,1627077471
197,Yangi Marg`ilon,40.4272,71.7189,78.22,27,0,2.01,UZ,1627077701
220,Córdoba,-31.4135,-64.1811,76.05,31,0,5.75,AR,1627077508
280,Muğla Province,37.2303,28.3557,72.27,45,0,6.24,TR,1627077614
306,Maceió,-9.6658,-35.7353,76.44,73,0,5.75,BR,1627077670
389,Aksu,41.1231,80.2644,76.71,51,0,5.28,CN,1627077484
399,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,74.84,28,0,7.14,AR,1627077782
412,Waingapu,-9.6567,120.2641,70.25,75,0,6.08,ID,1627077787
524,Vilhena,-12.7406,-60.1458,76.84,38,0,3.76,BR,1627077833


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
48,Jadu,LY,31.9530,12.0261,
171,Zhezkazgan,KZ,47.8043,67.7144,
197,Yangi Marg`ilon,UZ,40.4272,71.7189,
220,Córdoba,AR,-31.4135,-64.1811,
280,Muğla Province,TR,37.2303,28.3557,
306,Maceió,BR,-9.6658,-35.7353,
389,Aksu,CN,41.1231,80.2644,
399,Presidencia Roque Sáenz Peña,AR,-26.7852,-60.4388,
412,Waingapu,ID,-9.6567,120.2641,
524,Vilhena,BR,-12.7406,-60.1458,


In [8]:
#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 48: Jadu.
Missing field/result... skipping.
------------
Retrieving Results for Index 171: Zhezkazgan.
Closest hotel in Zhezkazgan is Cosmos.
------------
Retrieving Results for Index 197: Yangi Marg`ilon.
Closest hotel in Yangi Marg`ilon is TANTANA HOTEL FERGANA.
------------
Retrieving Results for Index 220: Córdoba.
Closest hotel in Córdoba is Amérian Executive Córdoba Hotel.
------------
Retrieving Results for Index 280: Muğla Province.
Closest hotel in Muğla Province is TN&CO Exclusive CIP Suites/Primeclass Rooms.
------------
Retrieving Results for Index 306: Maceió.
Closest hotel in Maceió is Jatiúca Hotel & Resort.
------------
Retrieving Results for Index 389: Aksu.
Closest hotel in Aksu is Tianyuan International Hotel.
------------
Retrieving Results for Index 399: Presidencia Roque Sáenz Peña.
Closest hotel in Presidencia Roque Sáenz Peña is Hotel Gualok.
------------
Retrieving Results for Index 412: Waingapu.
Closest hotel in Waingapu is Padadi

In [10]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
48,Jadu,LY,31.9530,12.0261,
171,Zhezkazgan,KZ,47.8043,67.7144,Cosmos
197,Yangi Marg`ilon,UZ,40.4272,71.7189,TANTANA HOTEL FERGANA
220,Córdoba,AR,-31.4135,-64.1811,Amérian Executive Córdoba Hotel
280,Muğla Province,TR,37.2303,28.3557,TN&CO Exclusive CIP Suites/Primeclass Rooms
306,Maceió,BR,-9.6658,-35.7353,Jatiúca Hotel & Resort
389,Aksu,CN,41.1231,80.2644,Tianyuan International Hotel
399,Presidencia Roque Sáenz Peña,AR,-26.7852,-60.4388,Hotel Gualok
412,Waingapu,ID,-9.6567,120.2641,Padadita Beach Hotel
524,Vilhena,BR,-12.7406,-60.1458,Portinari Palace Hotel


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))